### IMPORT DES BIBLIOTHÉQUES ET INITIALISATION DES OPTIONS

In [413]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
import re
import time

In [414]:
pd.set_option('display.max_columns',None)

In [415]:
load_dotenv()

True

### Réccupération des données de Festival de France avec l'API de DATA.GOUV

In [ ]:
dataset_id = "festivals-global-festivals-_-pl"
api_key = os.getenv("API_KEY")  
url = f"https://data.culture.gouv.fr/api/v2/catalog/datasets/{dataset_id}/exports/json"

headers = {
    'X-API-KEY': api_key
}

response = requests.get(url, headers=headers)
print("Code d'état:", response.status_code)
print("Contenu de la réponse", response.text)

if response.status_code == 200:
    try:
        data = response.json()

        print(json.dumps(data, indent=1))
        df = pd.DataFrame(data)
        csv_filename = 'festivals_data.csv'
        df.to_csv(csv_filename, index=False)
    except json.JSONDecodeError as e:
        print("Erreur de décodage JSON:", e)
else:
    print("Échec de la requête. Code d'état:", response.status_code)

### EXPLORATION DU JEU DE DONNÉES

In [417]:
#df = pd.read_csv("festivals_data.csv")


In [418]:
df.head(5)

,nom_du_festival,envergure_territoriale,region_principale_de_deroulement,departement_principal_de_deroulement,commune_principale_de_deroulement,code_postal_de_la_commune_principale_de_deroulement,code_insee_commune,code_insee_epci_collage_en_valeur,libelle_epci_collage_en_valeur,numero_de_voie,type_de_voie_rue_avenue_boulevard_etc,nom_de_la_voie,adresse_postale,complement_d_adresse_facultatif,site_internet_du_festival,adresse_e_mail,decennie_de_creation_du_festival,annee_de_creation_du_festival,discipline_dominante,sous_categorie_spectacle_vivant,sous_categorie_musique,sous_categorie_musique_cnm,sous_categorie_cinema_et_audiovisuel,sous_categorie_arts_visuels_et_arts_numeriques,sous_categorie_livre_et_litterature,periode_principale_de_deroulement_du_festival,identifiant_agence_a,identifiant,geocodage_xy,identifiant_cnm
0,Les Nuits (Re)Belles,None,Bourgogne-Franche-Comté,Jura,Brainans,39800,39073,200071595,"CC Arbois, Poligny, Salins, Cœur du Jura",None,None,None,None,None,http://www.moulindebrainans.com/wp-content/upl...,None,2010 et après,2014,Musique,None,None,07- Musiques actuelles sans distinction,None,None,None,Saison (21 juin - 5 septembre),None,FEST_39073_1020,"{'lon': 5.6278782462, 'lat': 46.8746666632}",2980.0
1,Y'a pas l'feu,Régionale,Hauts-de-France,Pas-de-Calais,Cambrin,62149,62200,200072460,"CA de Béthune-Bruay, Artois-Lys Romane",94,Boulevard,Louis Lesage,94 Boulevard Louis Lesage,None,www.festival-yapaslfeu.com,03 21 49 21 21,2010 et après,2014,Musique,None,"Rock, Métal",02- Musiques amplifiées ou électroniques,None,None,None,Avant-saison (1er janvier - 20 juin),None,FEST_62200_1194,"{'lon': 2.73659179875, 'lat': 50.5116047377}",2156.0
2,Festival du jeune art russe,None,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Cannes,6414,6004,240600585,CA de Sophia Antipolis,None,None,None,None,None,www.palaisdesfestivals.com,sortiracannes@palaisdesfestivals.com,2010 et après,2019,Musique,None,Musique traditionnelle,None,None,None,None,Saison (21 juin - 5 septembre),None,FEST_06004_1222,"{'lon': 7.10635418256, 'lat': 43.587465146}",NaN
3,Festival de musique de Cordes-sur-Ciel,None,Occitanie,Tarn,Cordes-sur-Ciel,81170,81069,200034064,CC du Cordais et du Causse (4 C),40,Rue,Grand rue raimond VII,40 Rue Grand rue raimond VII,Maison Gaugiran,https://www.festivalcordessurciel.com/,acadoc@wanadoo.fr,Avant 1980,1971,Musique,None,Musique classique,"11- Musique classique, lyrique, contemporaine,...",None,None,None,Saison (21 juin - 5 septembre),None,FEST_81069_1818,"{'lon': 1.95360871103, 'lat': 44.0681683828}",5932.0
4,Les rencontres de la cinémathèque montagne,None,Provence-Alpes-Côte d'Azur,Hautes-Alpes,Gap,5000,5061,200067825,CA Gap-Tallard-Durance,None,None,None,None,None,www.cimalpes.fr,cim@cimalpes.fr,De 2000 à 2009,2009,"Cinéma, audiovisuel",None,None,None,None,None,None,Après-saison (6 septembre - 31 décembre),None,FEST_05061_2353,"{'lon': 6.06486052138, 'lat': 44.5798600596}",NaN


In [419]:
df.columns.tolist()

['nom_du_festival',
 'envergure_territoriale',
 'region_principale_de_deroulement',
 'departement_principal_de_deroulement',
 'commune_principale_de_deroulement',
 'code_postal_de_la_commune_principale_de_deroulement',
 'code_insee_commune',
 'code_insee_epci_collage_en_valeur',
 'libelle_epci_collage_en_valeur',
 'numero_de_voie',
 'type_de_voie_rue_avenue_boulevard_etc',
 'nom_de_la_voie',
 'adresse_postale',
 'complement_d_adresse_facultatif',
 'site_internet_du_festival',
 'adresse_e_mail',
 'decennie_de_creation_du_festival',
 'annee_de_creation_du_festival',
 'discipline_dominante',
 'sous_categorie_spectacle_vivant',
 'sous_categorie_musique',
 'sous_categorie_musique_cnm',
 'sous_categorie_cinema_et_audiovisuel',
 'sous_categorie_arts_visuels_et_arts_numeriques',
 'sous_categorie_livre_et_litterature',
 'periode_principale_de_deroulement_du_festival',
 'identifiant_agence_a',
 'identifiant',
 'geocodage_xy',
 'identifiant_cnm']

### SUPPRESSION DES COLONNES INUTILES

In [ ]:
df.duplicated().unique()

In [420]:
df = df.drop([
 'envergure_territoriale',
 'code_insee_epci_collage_en_valeur',
 'libelle_epci_collage_en_valeur',
 'adresse_e_mail',
 'decennie_de_creation_du_festival',
 'identifiant_agence_a',
 'identifiant',
 'adresse_postale',
 'identifiant_cnm',], axis=1)

In [421]:
df.columns

Index(['nom_du_festival', 'region_principale_de_deroulement',
       'departement_principal_de_deroulement',
       'commune_principale_de_deroulement',
       'code_postal_de_la_commune_principale_de_deroulement',
       'code_insee_commune', 'numero_de_voie',
       'type_de_voie_rue_avenue_boulevard_etc', 'nom_de_la_voie',
       'complement_d_adresse_facultatif', 'site_internet_du_festival',
       'annee_de_creation_du_festival', 'discipline_dominante',
       'sous_categorie_spectacle_vivant', 'sous_categorie_musique',
       'sous_categorie_musique_cnm', 'sous_categorie_cinema_et_audiovisuel',
       'sous_categorie_arts_visuels_et_arts_numeriques',
       'sous_categorie_livre_et_litterature',
       'periode_principale_de_deroulement_du_festival', 'geocodage_xy'],
      dtype='object')

### CREATION D'UN NOUVEAU JEU DE DONNÉES 

In [422]:
# Renommer certaines colonnes
df_renomme = df.rename(columns={
    'nom_du_festival': 'Nom_Festival',
    'region_principale_de_deroulement': 'Region',
    'departement_principal_de_deroulement': 'Departement',
    'commune_principale_de_deroulement': 'Commune',
    'code_insee_commune': 'Code_INSEE',
    'site_internet_du_festival': 'Site_Internet',
    'annee_de_creation_du_festival': 'Annee_Creation',
    'discipline_dominante': 'Discipline_Principale',
    'periode_principale_de_deroulement_du_festival': 'Periode',
    'geocodage_xy': 'Geocode'
})

# Convertir toutes les valeurs en chaînes de caractères avant de les combiner
colonnes_adresse = ['numero_de_voie', 'type_de_voie_rue_avenue_boulevard_etc', 
                    'nom_de_la_voie', 'code_postal_de_la_commune_principale_de_deroulement', 
                    'complement_d_adresse_facultatif']

df_renomme[colonnes_adresse] = df_renomme[colonnes_adresse].fillna('').astype(str)

# Créer la colonne 'Adresse_Postale'
df_renomme['Adresse_Postale'] = df_renomme[colonnes_adresse].agg(' '.join, axis=1)

# Regrouper les sous-catégories en une seule colonne 'Sous_Categorie'
colonnes_sous_categorie = ['sous_categorie_spectacle_vivant', 'sous_categorie_musique',
                           'sous_categorie_musique_cnm', 'sous_categorie_cinema_et_audiovisuel',
                           'sous_categorie_arts_visuels_et_arts_numeriques',
                           'sous_categorie_livre_et_litterature']

df_renomme[colonnes_sous_categorie] = df_renomme[colonnes_sous_categorie].fillna('').astype(str)

df_renomme['Sous_Categorie'] = df_renomme[colonnes_sous_categorie].agg(' '.join, axis=1)

# Sélectionner uniquement les colonnes nécessaires pour le nouveau DataFrame
colonnes_selectionnees = [
    'Nom_Festival', 'Region', 'Departement', 'Commune', 'Adresse_Postale', 
    'Code_INSEE', 'Site_Internet', 'Annee_Creation', 
    'Discipline_Principale', 'Sous_Categorie', 'Periode', 'Geocode'
]


df_festival = df_renomme[colonnes_selectionnees]



In [423]:
df_festival.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7283 entries, 0 to 7282
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Nom_Festival           7283 non-null   object
 1   Region                 7283 non-null   object
 2   Departement            7257 non-null   object
 3   Commune                7257 non-null   object
 4   Adresse_Postale        7283 non-null   object
 5   Code_INSEE             7283 non-null   object
 6   Site_Internet          6558 non-null   object
 7   Annee_Creation         5835 non-null   object
 8   Discipline_Principale  7283 non-null   object
 9   Sous_Categorie         7283 non-null   object
 10  Periode                7223 non-null   object
 11  Geocode                7250 non-null   object
dtypes: object(12)
memory usage: 682.9+ KB


### TRAITEMENT DES COLONNES

**Colonnes GeoCode**

In [424]:
df_festival['Latitude'] = df_festival['Geocode'].apply(lambda x: x['lat'] if x is not None else None)
df_festival['Longitude'] = df_festival['Geocode'].apply(lambda x: x['lon'] if x is not None else None)
df_festival.drop('Geocode', axis=1, inplace=True)
df_festival.head(5)

/tmp/ipykernel_4065/2220115488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Latitude'] = df_festival['Geocode'].apply(lambda x: x['lat'] if x is not None else None)
/tmp/ipykernel_4065/2220115488.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Longitude'] = df_festival['Geocode'].apply(lambda x: x['lon'] if x is not None else None)
/tmp/ipykernel_4065/2220115488.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

,Nom_Festival,Region,Departement,Commune,Adresse_Postale,Code_INSEE,Site_Internet,Annee_Creation,Discipline_Principale,Sous_Categorie,Periode,Latitude,Longitude
0,Les Nuits (Re)Belles,Bourgogne-Franche-Comté,Jura,Brainans,39800,39073,http://www.moulindebrainans.com/wp-content/upl...,2014,Musique,07- Musiques actuelles sans distinction,Saison (21 juin - 5 septembre),46.874667,5.627878
1,Y'a pas l'feu,Hauts-de-France,Pas-de-Calais,Cambrin,94 Boulevard Louis Lesage 62149,62200,www.festival-yapaslfeu.com,2014,Musique,"Rock, Métal 02- Musiques amplifiées ou électr...",Avant-saison (1er janvier - 20 juin),50.511605,2.736592
2,Festival du jeune art russe,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Cannes,6414,6004,www.palaisdesfestivals.com,2019,Musique,Musique traditionnelle,Saison (21 juin - 5 septembre),43.587465,7.106354
3,Festival de musique de Cordes-sur-Ciel,Occitanie,Tarn,Cordes-sur-Ciel,40 Rue Grand rue raimond VII 81170 Maison Gaug...,81069,https://www.festivalcordessurciel.com/,1971,Musique,"Musique classique 11- Musique classique, lyri...",Saison (21 juin - 5 septembre),44.068168,1.953609
4,Les rencontres de la cinémathèque montagne,Provence-Alpes-Côte d'Azur,Hautes-Alpes,Gap,5000,5061,www.cimalpes.fr,2009,"Cinéma, audiovisuel",,Après-saison (6 septembre - 31 décembre),44.579860,6.064861


**Colonne Anne_Creation**

In [425]:
df_festival['Annee_Creation'].unique()

array(['2014', '2019', '1971', '2009', '1998', '1993', '2011', None,
       '2018', '2015', '2003', '1984', '1983', '1995', '2005', '2017',
       '1974', '2010', '2004', '1999', '2013', '2001', '1989', '2007',
       '01/01/2005 00:00', '2000', '01/01/2017 00:00', '26ème', '1992',
       '2016', '2002', '2008', '01/01/2004 00:00', '1990', '2012', '1969',
       '1987', '01/01/2012 00:00', '1997', '01/01/2014 00:00', '1991',
       '2021', '1994', '01/01/1996 00:00', '01/01/2011 00:00',
       '02/01/2015 00:00', '01/01/2018 00:00', '1972', '01/09/1998 00:00',
       '01/01/2009 00:00', '01/01/2010 00:00', '02/01/2009 00:00',
       '01/01/2019 00:00', '1982', '1980', '02/01/2006 00:00',
       '01/01/2016 00:00', '01/01/2008 00:00', '1976', '2006', '18ème',
       '27/06/1905 00:00', '01/01/1999 00:00', '1996', '1978', '2020',
       '1988', '1979', '1981', '1973', '1948', '1975', '19ème', '1986',
       '1985', '1604', '01/01/1956 00:00', '02/01/1994 00:00', '2004 ?',
       '1874', 

In [426]:
def extraire_annee(annee_str):
    if pd.isnull(annee_str):
        return None
    annee_str = str(annee_str)
    # Essayer de trouver une année à 4 chiffres
    match = re.search(r'\b(19|20)\d{2}\b', annee_str)
    if match:
        return int(match.group(0))
    # Essayer de trouver une année à 4 chiffres avec d'autres caractères
    match = re.search(r'\b(19|20)\d{2}', annee_str)
    if match:
        return int(match.group(0))
    # Trouver une année à 2 chiffres (e.g., 12ème en 21)
    match = re.search(r'(\d{1,2})\D*en\s(\d{2})', annee_str)
    if match:
        return 2021 - int(match.group(1))
    # Trouver des années à 2 chiffres dans des formats "XXème"
    match = re.search(r'(\d{1,2})\D*ème', annee_str)
    if match:
        return 2021 - int(match.group(1))
    # Trouver des années au format "17ans"
    match = re.search(r'(\d{1,2})\s?ans', annee_str)
    if match:
        return 2021 - int(match.group(1))
    # Trouver des années séparées par des slashs et conserver la première année
    match = re.search(r'(\d{4})\s?/\s?(\d{4})', annee_str)
    if match:
        return int(match.group(1))
    return None

# Appliquer la fonction à la colonne Annee_Creation
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(extraire_annee).astype('Int64')


/tmp/ipykernel_4065/1852256272.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(extraire_annee).astype('Int64')


In [427]:
df_festival['Annee_Creation'].unique()

<IntegerArray>
[2014, 2019, 1971, 2009, 1998, 1993, 2011, <NA>, 2018, 2015, 2003, 1984, 1983,
 1995, 2005, 2017, 1974, 2010, 2004, 1999, 2013, 2001, 1989, 2007, 2000, 1992,
 2016, 2002, 2008, 1990, 2012, 1969, 1987, 1997, 1991, 2021, 1994, 1996, 1972,
 1982, 1980, 2006, 1976, 1905, 1978, 2020, 1988, 1979, 1981, 1973, 1948, 1975,
 1986, 1985, 1956, 2022, 1977, 1946, 1950, 1968, 1920, 1966, 1934, 1967, 1958,
 1962, 1964, 1930, 1947, 1954, 1961, 1952, 1923, 1970, 1957, 1960, 1944, 1949,
 1965, 1951, 1935, 1959]
Length: 82, dtype: Int64

**Colonne Sous-Categorie**

In [428]:
df_festival["Sous_Categorie"].unique().tolist()

['  07- Musiques actuelles sans distinction   ',
 ' Rock, Métal 02- Musiques amplifiées ou électroniques   ',
 ' Musique traditionnelle    ',
 ' Musique classique 11- Musique classique, lyrique, contemporaine, autres   ',
 '     ',
 ' Musiques actuelles 03- Jazz, blues et musiques improvisées   ',
 ' Pop, rock 02- Musiques amplifiées ou électroniques   ',
 ' Hip-hop, rap, slam, Musiques du monde, Musiques électroniques, techno, Pop, rock, RnB 07- Musiques actuelles sans distinction   ',
 '     Pas de sous catégorie',
 ' Musiques classiques 11- Musique classique, lyrique, contemporaine, autres   ',
 '    Arts numériques, Installation, Performance ',
 ' Chanson ou variété française, Musiques du monde, Musiques traditionnelles, Musiques électroniques, techno, Hip-hop, rap, slam, Jazz, blues, spectacle pour enfants 09- Pluridisciplinaire   ',
 ' Musiques du monde, Reggae 02- Musiques amplifiées ou électroniques   ',
 ' 03- Jazz, blues et musiques improvisées 03- Jazz, blues et musiques imp

In [429]:
def clean_sous_categorie(value):
    # Supprimer les numéros suivis d'un tiret et d'un espace
    value = re.sub(r'\d+\s*-\s*', '', value)
    # Supprimer les espaces au début et à la fin
    value = value.strip()
    # Remplacer les valeurs vides par 'Inconnu' 
    if not value:
        return 'Inconnu'
    return value


df_festival['Sous_Categorie'] = df_festival['Sous_Categorie'].apply(clean_sous_categorie)


/tmp/ipykernel_4065/2955317777.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Sous_Categorie'] = df_festival['Sous_Categorie'].apply(clean_sous_categorie)


**Colonne Period**
 - Creation d'une nouvelle colonne 'Categorie_Periode'

In [430]:
def categoriser_periode(periode):
    if pd.isna(periode):
        return "Inconnu"
    periode = periode.lower()
    if 'avant-saison' in periode:
        return 'Avant-saison'
    elif any(mois in periode for mois in ['janvier', 'février', 'mars', 'avril','mai']):
        return 'Avant-saison'
    elif 'saison' in periode:
        return 'Saison'
    elif any(mois in periode for mois in ['juin', 'juillet', 'août', 'septembre']):
        return 'Saison'
    elif 'après-saison' in periode:
        return 'Après-saison'
    elif any(mois in periode for mois in ['octobre', 'novembre', 'décembre']):
        return 'Après-saison'
    elif 'janvier, février, mars, avril, mai, juin, juillet, août' in periode:
        return 'Période Variable'
    else:
        return 'Période Variable'

df_festival['Categorie_Periode'] = df_festival['Periode'].apply(categoriser_periode)


/tmp/ipykernel_4065/4238049019.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Categorie_Periode'] = df_festival['Periode'].apply(categoriser_periode)


In [431]:
def uniformiser_periode(event):
    if event is None:
        return None
    # Capitalize first letter of each word
    event = ' '.join(word.capitalize() for word in event.split())
    
    # Remove patterns 'avant saison', 'saison', 'apres-saison' and parentheses
    event = event.replace('Avant-saison', '').replace('Saison', '').replace('Après-saison', '').replace('(', '').replace(')', '') 
    
    return event.strip()

df_festival['Periode'] = df_festival['Periode'].apply(uniformiser_periode())

/tmp/ipykernel_4065/2608202292.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Periode'] = df_festival['Periode'].apply(clean_event_name)


In [432]:
df_festival['Periode'].unique().tolist()

['21 Juin - 5 Septembre',
 '1er Janvier - 20 Juin',
 '6 Septembre - 31 Décembre',
 None,
 'Novembre',
 'Mars',
 'Juillet',
 'Septembre',
 'Octobre',
 'Juin',
 'Août',
 'Février',
 'Décembre',
 'Avril',
 'Mai',
 'Période Variable Selon Les Territoires',
 'Janvier, Février, Mars, Avril, Mai, Juin, Juillet, Août',
 'Ocotbre',
 'Variable Selon Les Années',
 'Janvier, Octobre, Novembre']

In [436]:
df_festival

,Nom_Festival,Region,Departement,Commune,Adresse_Postale,Code_INSEE,Site_Internet,Annee_Creation,Discipline_Principale,Sous_Categorie,Periode,Latitude,Longitude,Categorie_Periode
0,Les Nuits (Re)Belles,Bourgogne-Franche-Comté,Jura,Brainans,39800,39073,http://www.moulindebrainans.com/wp-content/upl...,2014,Musique,Musiques actuelles sans distinction,21 Juin - 5 Septembre,46.874667,5.627878,Saison
1,Y'a pas l'feu,Hauts-de-France,Pas-de-Calais,Cambrin,94 Boulevard Louis Lesage 62149,62200,www.festival-yapaslfeu.com,2014,Musique,"Rock, Métal Musiques amplifiées ou électroniques",1er Janvier - 20 Juin,50.511605,2.736592,Avant-saison
2,Festival du jeune art russe,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Cannes,6414,6004,www.palaisdesfestivals.com,2019,Musique,Musique traditionnelle,21 Juin - 5 Septembre,43.587465,7.106354,Saison
3,Festival de musique de Cordes-sur-Ciel,Occitanie,Tarn,Cordes-sur-Ciel,40 Rue Grand rue raimond VII 81170 Maison Gaug...,81069,https://www.festivalcordessurciel.com/,1971,Musique,"Musique classique Musique classique, lyrique, ...",21 Juin - 5 Septembre,44.068168,1.953609,Saison
4,Les rencontres de la cinémathèque montagne,Provence-Alpes-Côte d'Azur,Hautes-Alpes,Gap,5000,5061,www.cimalpes.fr,2009,"Cinéma, audiovisuel",Inconnu,6 Septembre - 31 Décembre,44.579860,6.064861,Saison
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7278,Les Z'arpetes,Nouvelle-Aquitaine,Gironde,Villenave-d'Ornon,Chemin de Cadaujac 33140 Complexe de la Junca,33550,http://www.leszarpetes.com/,2003,Spectacle vivant,Spectacle vivant pluridisciplinaire,1er Janvier - 20 Juin,44.773185,-0.558212,Avant-saison
7279,Jazz à Ville sur Auzon,Provence-Alpes-Côte d'Azur,Vaucluse,Villes-sur-Auzon,84570,84148,jazzavillessurauzon.free.fr,2004,Musique,Jazz,21 Juin - 5 Septembre,44.066061,5.257078,Saison
7280,Parlez-moi d'humour,Bretagne,Ille-et-Vilaine,Vitré,Place du Château 35500,35360,www.mairie-vitre.com,2018,"Livre, littérature",Arts de la rue,21 Juin - 5 Septembre,48.114082,-1.193707,Saison
7281,Festival des cultures du monde,Auvergne-Rhône-Alpes,Isère,Voiron,45 Avenue de Paviot 38500,38563,https://www.salmorencvoironculturesdumonde.com...,1983,Spectacle vivant,Danse,21 Juin - 5 Septembre,45.379172,5.582403,Saison


**ADRESSE**

In [433]:
# def get_address_from_coordinates(lat, lon):
#     try:
#         url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"

#         time.sleep(1)

#         response = requests.get(url)

#         # Vérifiez si la requête a réussi
#         if response.status_code == 200:
#             json_data = response.json()

#             # Extraire l'adresse de la réponse JSON
#             if "address" in json_data:
#                 address = json_data["address"]

#                 # Construire l'adresse complète depuis les donénes de l'api
#                 full_address = ", ".join(filter(None, [
#                     address.get("road", ""),
#                     address.get("quarter", ""),
#                     address.get("city", ""),
#                     address.get("municipality", ""),
#                     address.get("county", ""),
#                     address.get("state", ""),
#                     address.get("region", ""),
#                     address.get("postcode", "")
#                 ]))

#                 return full_address
#     except Exception as e:
#         print(f"Error getting address: {e}")
#     return ""

# df_festival['Adresse_Complete'] = df_festival.apply(lambda row: get_address_from_coordinates(row['latitude'], row['longitude']) if not pd.isna(row['latitude']) and not pd.isna(row['longitude']) else "", axis=1)


In [434]:
#df_festival.rename(columns={'Adresse_Complete': 'Adresse_Postale'}, inplace=True)


In [435]:
#df_festival.to_csv('clean_Fest.csv', index=False)